# Set up the environment

In [ ]:
!pip install cmake 'gym[atari]' scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def environment(env_name = ""):
  import gym
  env = gym.make(env_name).env
  env.render()
  return env

In [ ]:
def env_states_actions(env):
  env.reset() # reset environment to a new, random state
  env.render()

  print("Action Space {}".format(env.action_space))
  print("State Space {}".format(env.observation_space))

# Brute force function


In [ ]:
def brute_force(env):
  env.s = 328  # set environment to illustration's state

  epochs = 0
  penalties, rewards = 0, 0

  frames = [] # for animation

  done = False

  while not done:
      action = env.action_space.sample()
      state, reward, done, info = env.step(action)

      if reward == -10:
          penalties += 1

      if reward > 0:
        rewards += 1

      # Put each rendered frame into dict for animation
      frames.append({
          'frame': env.render(mode='ansi'),
          'state': state,
          'action': action,
          'reward': reward
          }
      )

      epochs += 1


  return epochs, penalties, frames

In [ ]:
def print_frames(frames):
  from IPython.display import clear_output
  from time import sleep
  for i, frame in enumerate(frames):
      clear_output(wait=True)
      print(frame['frame'])
      print(f"Timestep: {i + 1}")
      print(f"State: {frame['state']}")
      print(f"Action: {frame['action']}")
      print(f"Reward: {frame['reward']}")
      # sleep(1)

#Q-Learning approach

#Training function with decay

In [ ]:
def training_decay(env):
  %%time

  import random
  from IPython.display import clear_output
  import numpy as np
  q_tables =  []

  # Initialize the q table
  q_table = np.zeros([env.observation_space.n, env.action_space.n])

  # For plotting metrics
  all_epochs = []
  all_penalties = []


  # Hyperparameters

  alpha = 0.9
  gamma = 0.9
  epsilon = 0.9

  for i in range(1, 100001):
      state = env.reset()

      epochs, penalties, reward, = 0, 0, 0
      done = False

      while not done:
          if random.uniform(0, 1) < epsilon:
              action = env.action_space.sample() # Explore action space
          else:
              action = np.argmax(q_table[state]) # Exploit learned values

          next_state, reward, done, info = env.step(action)

          old_value = q_table[state, action]
          next_max = np.max(q_table[next_state])

          new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
          q_table[state, action] = new_value

          if reward == -10:
              penalties += 1

          state = next_state
          epochs += 1

      if i % 20000 == 0:
          clear_output(wait=True)
          alpha = alpha*0.9
          gamma = gamma*0.9
          epsilon = epsilon*0.9
          print(f"Episode: {i}")

  q_tables.append(q_table.tolist())
  print("Training finished.\n")
  return q_tables

#Hyperparameters Combination

Function to create all possible combinations of the hyperparameters alpha, gamma and epsilon

In [ ]:
def hyperp_comb(alpha_list, gamma_list, epsilon_list):
  import itertools

  hyperp_comb = list(itertools.product(alpha_list, gamma_list, epsilon_list))
  return hyperp_comb

#Training function with grid search

In [ ]:
def training_grid(env, hyperp_comb = []):
  %%time

  import random
  from IPython.display import clear_output
  import numpy as np
  q_tables =  []

  # Initialize the q table
  q_table = np.zeros([env.observation_space.n, env.action_space.n])

  # For plotting metrics
  all_epochs = []
  all_penalties = []


  # Hyperparameters
  for j in hyperp_comb:
      alpha = j[0]
      gamma = j[1]
      epsilon = j[2]

      for i in range(1, 100001):
        state = env.reset()

        epochs, penalties, reward, = 0, 0, 0
        done = False

        while not done:
            if random.uniform(0, 1) < epsilon:
                action = env.action_space.sample() # Explore action space
            else:
                action = np.argmax(q_table[state]) # Exploit learned values

            next_state, reward, done, info = env.step(action)

            old_value = q_table[state, action]
            next_max = np.max(q_table[next_state])

            new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
            q_table[state, action] = new_value

            if reward == -10:
                penalties += 1

            state = next_state
            epochs += 1

        if i % 100 == 0:
            clear_output(wait=True)
            print(f"Episode: {i}")

      q_tables.append(q_table.tolist())

  print("Training finished.\n")
  return q_tables

# Evalutation

In [ ]:
def evaluation(env, q_tables, episodes):
  import numpy as np
  total_epochs, total_penalties = 0, 0
  number = 0
  for q_table in q_tables:
    number += 1
    for _ in range(episodes):
        state = env.reset()
        epochs, penalties, reward = 0, 0, 0

        done = False

        while not done:
            action = np.argmax(q_table[state])
            state, reward, done, info = env.step(action)

            if reward == -10:
                penalties += 1

            epochs += 1

        total_penalties += penalties
        total_epochs += epochs

    print(f"Number of q_table: {number}")
    print(f"Results after {episodes} episodes:")
    print(f"Average timesteps per episode: {total_epochs / episodes}")
    print(f"Average penalties per episode: {total_penalties / episodes}")


# Implementation

###Run the environment

In [ ]:
env = environment("Taxi-v3")

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [ ]:
env_states_actions(env)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


###Brute_force approach

In [ ]:
epochs,penalties, frames = brute_force(env)

print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 7343
Penalties incurred: 2489


In [ ]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 7343
State: 0
Action: 5
Reward: 20


##Decay

###Training

In [ ]:
q_tables = training_decay(env)

Episode: 100000
Training finished.



###Evaluation

In [ ]:
evaluation(env, q_tables, 1000)

Number of q_table: 1
Results after 1000 episodes:
Average timesteps per episode: 13.041
Average penalties per episode: 0.0


##Grid search

###Hyperparameters Combination

In [ ]:
alphalist = [0.5, 0.9]
gammalist = [0.5, 0.9]
epsilonlist = [0.5, 0.9]

hyperp_comb = hyperp_comb(alphalist, gammalist, epsilonlist)
print(hyperp_comb)

[(0.5, 0.5, 0.5), (0.5, 0.5, 0.9), (0.5, 0.9, 0.5), (0.5, 0.9, 0.9), (0.9, 0.5, 0.5), (0.9, 0.5, 0.9), (0.9, 0.9, 0.5), (0.9, 0.9, 0.9)]


###Training

In [ ]:
q_tables = training_grid(env, hyperp_comb)

Episode: 100000
Training finished.



In [ ]:
len(q_tables)

8

###Evaluation

In [ ]:
evaluation(env, q_tables, 1000)

Number of q_table: 1
Results after 1000 episodes:
Average timesteps per episode: 13.1
Average penalties per episode: 0.0
Number of q_table: 2
Results after 1000 episodes:
Average timesteps per episode: 26.129
Average penalties per episode: 0.0
Number of q_table: 3
Results after 1000 episodes:
Average timesteps per episode: 39.277
Average penalties per episode: 0.0
Number of q_table: 4
Results after 1000 episodes:
Average timesteps per episode: 52.414
Average penalties per episode: 0.0
Number of q_table: 5
Results after 1000 episodes:
Average timesteps per episode: 65.497
Average penalties per episode: 0.0
Number of q_table: 6
Results after 1000 episodes:
Average timesteps per episode: 78.624
Average penalties per episode: 0.0
Number of q_table: 7
Results after 1000 episodes:
Average timesteps per episode: 91.778
Average penalties per episode: 0.0
Number of q_table: 8
Results after 1000 episodes:
Average timesteps per episode: 105.037
Average penalties per episode: 0.0
